In [2]:
from dotenv import load_dotenv
import os
# .env에 정의한 환경변수를 메모리에 로딩
load_dotenv()

# OPENAI_API_KEY 값을 변수에 대입
api_key = os.environ["OPENAI_API_KEY"]

In [16]:
# Redis 환경 설정 값 변수에 대입

host= os.environ["host"]
port= int(os.environ["port"])
password= os.environ["password"]

# print(host)
# print(port)
# print(password)

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002973055A720>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002973055BC20>, root_client=<openai.OpenAI object at 0x00000297300C2C30>, root_async_client=<openai.AsyncOpenAI object at 0x000002973055A750>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))

### caching


- https://python.langchain.com/docs/how_to/llm_caching/
- https://python.langchain.com/api_reference/core/caches/langchain_core.caches.InMemoryCache.html


In [22]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache
from langchain_core.messages import HumanMessage

set_llm_cache(InMemoryCache())

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

message = [HumanMessage(content="서울 광장시장에서 가장 맛있는 길거리 음식은 무엇인가요?")]

In [23]:
%%time
response = llm.invoke(message)
print(response.content)

서울 광장시장에서 가장 맛있는 길거리 음식으로는 떡볶이나 김밥, 붕어빵, 어묵탕 등 다양한 종류의 음식을 즐길 수 있습니다. 특히 떡볶이와 어묵탕은 광장시장을 방문한 관광객들 사이에서 인기 있는 메뉴로 손꼽힙니다. 매콤하고 고소한 맛이 일품이라고 할 수 있죠. 광장시장을 방문하면 다양한 길거리 음식을 즐기며 현지의 맛을 경험해보시는 것을 추천드립니다.
CPU times: total: 15.6 ms
Wall time: 2.6 s


In [24]:
%%time
response = llm.invoke(message)
print(response.content)

서울 광장시장에서 가장 맛있는 길거리 음식으로는 떡볶이나 김밥, 붕어빵, 어묵탕 등 다양한 종류의 음식을 즐길 수 있습니다. 특히 떡볶이와 어묵탕은 광장시장을 방문한 관광객들 사이에서 인기 있는 메뉴로 손꼽힙니다. 매콤하고 고소한 맛이 일품이라고 할 수 있죠. 광장시장을 방문하면 다양한 길거리 음식을 즐기며 현지의 맛을 경험해보시는 것을 추천드립니다.
CPU times: total: 0 ns
Wall time: 0 ns


In [25]:
set_llm_cache(InMemoryCache())
llm_1 = ChatOpenAI(model="gpt-3.5-turbo-0125")

llm_2 = ChatOpenAI(model="gpt-3.5-turbo-0125", cache=False)

message = [HumanMessage(content="서울 광장시장에서 가장 맛있는 길거리 음식은 무엇인가요?")]

In [26]:
%%time
response = llm_1.invoke(message)
print(response.content)

서울 광장시장에서 가장 맛있는 길거리 음식은 떡볶이나 핫도그, 떡꼬치, 튀김 등이 인기가 많습니다. 특히 떡볶이는 매콤하고 고소한 맛이 일품이라 많은 사람들이 즐겨 먹는 대표적인 길거리 음식입니다. 또한 광장시장에서는 다양한 종류의 음식을 맛볼 수 있어서 많은 이들이 찾는 곳이기도 합니다. 광장시장을 방문하면 다양한 맛과 향을 경험할 수 있으니 여러 음식을 시도해보는 것을 추천드립니다.
CPU times: total: 0 ns
Wall time: 2.48 s


In [27]:
%%time
response = llm_2.invoke(message)
print(response.content)

서울 광장시장에서 가장 맛있는 길거리 음식으로는 떡볶이, 어묵, 순대, 떡꼬치, 라면 등이 유명합니다. 특히 떡볶이와 어묵은 광장시장에서 먹어보면 그 맛을 잘 느낄 수 있는 대표적인 길거리 음식이라고 할 수 있습니다. 또한 각종 분식류와 디저트도 다양하게 즐길 수 있으니 시장을 방문해보시면 좋을 것 같습니다.
CPU times: total: 0 ns
Wall time: 1.97 s


- https://python.langchain.com/api_reference/community/cache/langchain_community.cache.RedisSemanticCache.html


In [5]:
%pip -q install redis

Note: you may need to restart the kernel to use updated packages.


In [17]:
from redis import Redis

redis_db = Redis(
  host = host ,
  port = port, 
  password = password)

In [18]:
from langchain_core.globals import set_llm_cache
from langchain_community.cache import RedisCache
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


# radis db에 캐싱함
set_llm_cache(RedisCache( 
    redis_= redis_db
))

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

message1 = [HumanMessage(content="서울에서 가장 맛있는 길거리 음식은 5개를 소개해 주세요")]
message2 = [HumanMessage(content="서울에서 가장 맛있는 길거리 음식은 10개를 소개해 주세요")]

In [19]:
%%time
response1 = llm.invoke(message1)
print(response1.content)

1. 홍대 닭강정: 홍대입구역 근처에 위치한 이곳은 닭강정 전문점으로 유명하며, 바삭하고 달콤한 맛이 일품입니다.
2. 명동 꼬치구이: 명동 시장에서 파는 다양한 종류의 꼬치구이는 고기의 식감과 풍부한 양념이 일품이며, 현지인들도 자주 찾는 인기 메뉴입니다.
3. 이태원 타코: 이태원의 다양한 외국 음식 중에서도 타코는 특히 맛있는 길거리 음식으로 손꼽힙니다. 신선한 재료와 특별한 소스로 맛을 더해줍니다.
4. 강남 떡볶이: 강남역 근처의 떡볶이 전문점은 매콤하고 쫄깃한 떡과 맛깔나는 양념으로 손님들을 매료시킵니다.
5. 종로 붕어빵: 종로에 위치한 붕어빵 전문점은 바삭한 외피와 달콤한 팥앙금으로 유명하며, 한국 전통 간식을 맛볼 수 있는 곳입니다.
CPU times: total: 0 ns
Wall time: 73.2 ms


In [20]:
%%time
response2 = llm.invoke(message2)
print(response2.content)

1. 홍대 닭강정: 홍대입구역 근처에서 맛볼 수 있는 닭강정은 바삭하고 담백한 맛으로 인기가 많습니다.
2. 명동 닭발: 명동 시장에서 판매되는 닭발은 매콤한 양념과 쫄깃한 식감으로 유명합니다.
3. 이태원 토스트: 이태원의 유명한 토스트 가게에서는 다양한 종류의 토스트를 맛볼 수 있습니다.
4. 광장시장 붕어빵: 광장시장에서 파는 붕어빵은 바삭한 피부와 달콤한 팥 속으로 유명합니다.
5. 신촌 핫도그: 신촌역 근처에서 판매되는 핫도그는 크리스피한 튀김옷과 속이 꽉 찬 맛으로 사랑받고 있습니다.
6. 이대 떡볶이: 이대역 근처에 위치한 떡볶이 가게에서는 고소한 양념과 쫄깃한 떡으로 유명합니다.
7. 종로 붉은쌀떡: 종로의 전통시장에서 파는 붉은쌀떡은 고소하고 달콤한 맛으로 인기가 있습니다.
8. 강남 불고기버거: 강남역 근처에 위치한 불고기버거 전문점에서는 고기의 질감과 양념이 일품입니다.
9. 한남동 타코: 한남동의 타코 가게에서는 신선한 재료와 특별한 소스로 만든 타코를 즐길 수 있습니다.
10. 신림 김밥: 신림역 근처에 위치한 김밥가게에서는 신선한 재료와 푸짐한 양으로 맛있는 김밥을 즐길 수 있습니다.
CPU times: total: 0 ns
Wall time: 9 ms
